In [1]:
import pandas as pd
from pathlib import Path
import os
from collections import defaultdict

Проверка описания

In [19]:
desc = pd.read_csv("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data/Женщинам_Футболки&и&топы/descriptions.csv")

In [20]:
desc

,item,description
0,0.png,Представляем вам женскую базовую бельевую спор...
1,1.png,Хлопковый стильный короткий кроп-топ в рубчик ...
2,2.png,Кроп топ женский на тонких бретелях - незамени...
3,3.png,Футболка женская плотная свободного кроя оверс...
4,4.png,Футболка Хизер от бренда Jelika – универсальна...
5,5.png,Чёрная футболка superoversize с принтом выпол...
6,6.png,"Топ двойной длинный - это трендовая вещь, кото..."
7,7.png,Футболка женская оверсайз бренда ТЕЛОДВИЖЕНИЯ ...
8,8.png,Добро пожаловать в нашу студию печати WAKA WAK...
9,9.png,Футболка с округлым вырезом давно стала неотъе...


____

Сбор статистики по спаршенным фотографиям

In [4]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/data_971")

In [5]:
# Кол-во категорий
folders = os.listdir(root_dir)
len(folders)

476

In [6]:
df_dict = {col: [] for col in ["card", "feedbacks"]}

for cat in folders:
    path = root_dir / cat
    path_card = path / "card"
    path_feedbacks = path / "feedbacks"

    # кол-во загруженных фоток
    load_card = len(sorted(path_card.glob('*')))
    load_feedbacks = len(sorted(path_feedbacks.glob('*')))
    
    if load_feedbacks < 18:
        print(f"{load_feedbacks} Feedbacks in {cat}")

    df_dict["card"].append(load_card)
    df_dict["feedbacks"].append(load_feedbacks)

16 Feedbacks in Автотовары_OFFroad
15 Feedbacks in Автотовары_Аксессуары&в&салон&и&багажник_Канистры
15 Feedbacks in Автотовары_Аксессуары&в&салон&и&багажник_Термокружки&и&подстаканники
9 Feedbacks in Автотовары_Запчасти&для&лодок&и&катеров
6 Feedbacks in Автотовары_Запчасти&на&силовую&технику
13 Feedbacks in Автотовары_Инструменты_Другие&автоинструменты
17 Feedbacks in Автотовары_Инструменты_Ключи
15 Feedbacks in Автотовары_Инструменты_Крепежный&инструмент
17 Feedbacks in Автотовары_Инструменты_Тестеры
17 Feedbacks in Автотовары_Коврики
16 Feedbacks in Автотовары_Краски&и&грунтовки_Краски
11 Feedbacks in Автотовары_Краски&и&грунтовки_Краски&аэрозольные
15 Feedbacks in Автотовары_Краски&и&грунтовки_Эмали
16 Feedbacks in Автотовары_Мойки&высокого&давления&и&аксессуары_Аксессуары&и&запчасти&для&садовой&техники
9 Feedbacks in Автотовары_Мойки&высокого&давления&и&аксессуары_Аэраторы&и&скарификаторы&газонные
10 Feedbacks in Автотовары_Мойки&высокого&давления&и&аксессуары_Газонокосилки
12 Fe

In [7]:
df = pd.DataFrame(df_dict)
df.head()

,card,feedbacks
0,20,16
1,20,18
2,20,18
3,20,20
4,20,20


In [8]:
df.describe()

,card,feedbacks
count,476.0,476.000000
mean,20.0,16.142857
std,0.0,5.543756
min,20.0,0.000000
25%,20.0,15.000000
50%,20.0,19.000000
75%,20.0,20.000000
max,20.0,20.000000


In [9]:
df.feedbacks.value_counts()

feedbacks
20    151
19     90
18     57
17     31
15     24
16     18
0      18
13     13
14     12
1       8
11      8
2       7
10      7
12      7
3       6
9       6
5       4
8       3
6       3
7       2
4       1
Name: count, dtype: int64

In [10]:
df.card.value_counts()

card
20    476
Name: count, dtype: int64

___

Проверка на дубликаты ссылок

In [42]:
categories = pd.read_csv("wb_categories.csv")
categories.head()

,category_1,category_2,category_3,url
0,Женщинам,Блузки и рубашки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
1,Женщинам,Брюки,NaN,https://www.wildberries.ru/catalog/zhenshchina...
2,Женщинам,Верхняя одежда,NaN,https://www.wildberries.ru/catalog/zhenshchina...
3,Женщинам,"Джемперы, водолазки и кардиганы",NaN,https://www.wildberries.ru/catalog/zhenshchina...
4,Женщинам,Джинсы,NaN,https://www.wildberries.ru/catalog/zhenshchina...


In [43]:
folders_name = []

for i in range(len(categories)):
    if str(categories.iloc[i]["category_3"]) == "nan":
        if str(categories.iloc[i]["category_2"]) == "nan":
            cat = list(categories.iloc[i][:1])
        else:
            cat = list(categories.iloc[i][:2])
    else:
        cat = list(categories.iloc[i][:3])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder)

In [45]:
df_dupl = categories[categories.duplicated(['url'], keep=False)]
df_dupl

,category_1,category_2,category_3,url
207,Детям,Детская электроника,Автоэлектроника и навигация,https://www.wildberries.ru/catalog/elektronika...
208,Детям,Детская электроника,Гарнитуры и наушники,https://www.wildberries.ru/catalog/elektronika...
210,Детям,Детская электроника,Кабели и зарядные устройства,https://www.wildberries.ru/catalog/elektronika...
211,Детям,Детская электроника,Музыка и видео,https://www.wildberries.ru/catalog/elektronika...
214,Детям,Детская электроника,Развлечения и гаджеты,https://www.wildberries.ru/catalog/elektronika...
...,...,...,...,...
1337,Сад и дача,Садовая техника,Газонокосилки,https://www.wildberries.ru/catalog/dom/dachniy...
1338,Сад и дача,Садовая техника,Подметальные машины,https://www.wildberries.ru/catalog/dom/sad-i-d...
1339,Сад и дача,Садовая техника,Аэраторы и скарификаторы газонные,https://www.wildberries.ru/catalog/dom/dachniy...
1355,Сад и дача,Полив и водоснабжение,Насосы и помпы,https://www.wildberries.ru/catalog/dom/dachniy...


In [46]:
df_dupl.index

Index([ 207,  208,  210,  211,  214,  215,  218,  219,  221,  222,
       ...
       1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1355, 1365],
      dtype='int64', length=156)

In [55]:
duplicates = defaultdict(list)

for ind in df_dupl.index:
    url = df_dupl.loc[ind]["url"]
    duplicates[url].append(ind)

In [60]:
print(len(duplicates))
duplicates

69


defaultdict(list,
            {'https://www.wildberries.ru/catalog/elektronika/avtoelektronika': [207,
              976],
             'https://www.wildberries.ru/catalog/elektronika/igry-i-razvlecheniya/aksessuary/garnitury': [208,
              651],
             'https://www.wildberries.ru/catalog/elektronika/kabeli-i-zaryadnye-ustroystva': [210,
              624],
             'https://www.wildberries.ru/catalog/elektronika/muzyka-i-video': [211,
              625],
             'https://www.wildberries.ru/catalog/elektronika/razvlecheniya-i-gadzhety': [214,
              640],
             'https://www.wildberries.ru/catalog/elektronika/setevoe-oborudovanie': [215,
              641],
             'https://www.wildberries.ru/catalog/elektronika/smart-chasy': [218,
              665],
             'https://www.wildberries.ru/catalog/elektronika/solnechnye-elektrostantsii-i-komplektuyushchie': [219,
              666],
             'https://www.wildberries.ru/catalog/elektronika/t

In [59]:
categories["url"][207], categories["url"][976]

('https://www.wildberries.ru/catalog/elektronika/avtoelektronika',
 'https://www.wildberries.ru/catalog/elektronika/avtoelektronika')

In [24]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]
})
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [25]:
df[df.duplicated(['style'], keep=False)]

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0
